In [1]:
import pandas as pd
import os
import glob
from dbfread import DBF


In [2]:
csv_directory = "./데이터/고도/"
csv_files = glob.glob(csv_directory + "*.csv")

In [3]:
locals = ["종로구", "중구", "용산구", "성동구", "광진구", "동대문구", "중랑구", "성북구", "강북구", "도봉구", "노원구", "은평구", "서대문구", "마포구", "양천구", "강서구", "구로구", "금천구", "영등포구", "동작구", "관악구", "서초구", "강남구", "송파구", "강동구"]


In [4]:
elevation = pd.DataFrame()

for filename in os.listdir(csv_directory):
    if filename.endswith(".csv"):
        file_path = os.path.join(csv_directory, filename)
        
        # CSV 파일을 읽어옵니다. 열 이름을 자동으로 생성하려면 header=None을 설정합니다.
        df = pd.read_csv(file_path, header=None)
        
        # 읽어온 데이터프레임을 병합합니다.
        elevation = pd.concat([elevation, df], ignore_index=True)


In [5]:
elevation.rename(columns={0: 'AREA', 1: "PERIMETER",2:'ELEVATION'}, inplace = True)

In [6]:
elevation

,AREA,PERIMETER,ELEVATION
0,200305.31,547459.91,16.78
1,200305.31,547369.91,9.62
2,200305.31,547279.91,5.69
3,200305.31,547189.91,5.38
4,200395.31,547549.91,19.62
...,...,...,...
106712,210697.72,554846.65,132.81
106713,210697.72,554756.65,121.28
106714,210697.72,554666.65,113.31
106715,210787.72,555026.65,146.72


In [7]:
elevation.to_csv('elevation.csv')

In [8]:
drainage= pd.read_excel("./데이터/Z_SIS_ASIT_SOILDRA_AREA.xlsx")


In [9]:
drainage.head()

,AREA,PERIMETER,CODE_DC,SOILDRA
0,20154500.0,26673.40,999,기타
1,1210520.0,5359.01,1,매우양호
2,187981.0,3806.03,2,양호
3,3612200.0,22563.10,2,양호
4,102804.0,4164.06,999,기타


In [10]:
type(drainage['AREA'])

pandas.core.series.Series

In [31]:
import pyproj
import folium
import numpy as np

In [45]:
elevation['AREA'] = pd.to_numeric(elevation['AREA'], errors="coerce")
elevation['PERIMETER'] = pd.to_numeric(elevation['PERIMETER'], errors="coerce")

elevation = elevation.dropna()
elevation.index=range(len(elevation))
elevation.tail()


C:\Users\yis82\AppData\Local\Temp\ipykernel_32620\188173397.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elevation['AREA'] = pd.to_numeric(elevation['AREA'], errors="coerce")
C:\Users\yis82\AppData\Local\Temp\ipykernel_32620\188173397.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elevation['PERIMETER'] = pd.to_numeric(elevation['PERIMETER'], errors="coerce")


,AREA,PERIMETER,ELEVATION
106711,210697.72,554846.65,132.81
106712,210697.72,554756.65,121.28
106713,210697.72,554666.65,113.31
106714,210787.72,555026.65,146.72
106715,210787.72,554936.65,126.35


In [46]:
def project_array(coord, p1_type, p2_type):

    p1 = pyproj.Proj(init=p1_type)
    p2 = pyproj.Proj(init=p2_type)
    fx, fy = pyproj.transform(p1, p2, coord[:, 0], coord[:, 1])
    return np.dstack([fx, fy])[0]


In [47]:
coord = np.array(elevation)
coord

array([[2.0030531e+05, 5.4745991e+05, 1.6780000e+01],
       [2.0030531e+05, 5.4736991e+05, 9.6200000e+00],
       [2.0030531e+05, 5.4727991e+05, 5.6900000e+00],
       ...,
       [2.1069772e+05, 5.5466665e+05, 1.1331000e+02],
       [2.1078772e+05, 5.5502665e+05, 1.4672000e+02],
       [2.1078772e+05, 5.5493665e+05, 1.2635000e+02]])

In [57]:
p1_type = "EPSG:5181"
p2_type = "EPSG:5186"

result = project_array(coord, p1_type, p2_type)
result = pd.DataFrame(result,columns=['AREA',"PERIMETER"])
result

C:\Users\yis82\AppData\Local\Temp\ipykernel_32620\43789815.py:5: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  fx, fy = pyproj.transform(p1, p2, coord[:, 0], coord[:, 1])


,AREA,PERIMETER
0,300305.31,547459.91
1,300305.31,547369.91
2,300305.31,547279.91
3,300305.31,547189.91
4,300395.31,547549.91
...,...,...
106711,310697.72,554846.65
106712,310697.72,554756.65
106713,310697.72,554666.65
106714,310787.72,555026.65


In [59]:
df = drainage.loc[drainage['AREA'].isin(result['AREA'])]


,AREA,PERIMETER,CODE_DC,SOILDRA


In [65]:
drainage.loc[drainage['AREA'] ==300305.3]

,AREA,PERIMETER,CODE_DC,SOILDRA
